In [73]:
from copy import deepcopy

import numpy as np
import open3d as o3d
from open3d.examples.pipelines.colored_icp_registration import target

In [42]:
mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\data\Dancer\meshes/dancer_fr0010.obj')
print(mesh)
testmesh = o3d.geometry.TriangleMesh()
testmesh.vertices = mesh.vertices
testmesh.triangles = mesh.triangles
testmesh.compute_vertex_normals()

print(testmesh.has_textures())
print(testmesh)
#o3d.visualization.draw_geometries([testmesh])

TriangleMesh with 20913 points and 39357 triangles.
False
TriangleMesh with 20913 points and 39357 triangles.


In [97]:
decimated_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(testmesh, 10000, boundary_weight= 8000)
print(decimated_mesh)
decimated_mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([decimated_mesh])

TriangleMesh with 6168 points and 10000 triangles.


simplify_vertex_clustering is not good for subdivide surface fitting, simplify_quadric_decimation is much better
notes that boundary_weight will influence the result of decimated mesh, inappropriate setting leads to holes in the mesh.

In [105]:
def subdivide_surface_fitting(decimated_mesh, target_mesh, iterations=1):
    subdivided_mesh = o3d.geometry.TriangleMesh()
    subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decimated_mesh, number_of_iterations=iterations)
    print(subdivided_mesh)
    subdivided_mesh.compute_vertex_normals()
    #o3d.visualization.draw_geometries([subdivided_mesh])
    
    pcd_target = o3d.geometry.PointCloud()
    pcd_target.points = o3d.utility.Vector3dVector(target_mesh.vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_target)
    subdivided_vertices = np.array(subdivided_mesh.vertices)
    target_vertices = np.array(target_mesh.vertices)
    fitting_vertices = deepcopy(subdivided_vertices)
    
    for i in range(0, len(subdivided_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(subdivided_vertices[i], 1)
        fitting_vertices[i] = target_vertices[np.asarray(index)]
        
    subdivided_mesh.vertices = o3d.utility.Vector3dVector(fitting_vertices)
    return subdivided_mesh

In [99]:
fitting_mesh = subdivide_surface_fitting(decimated_mesh, mesh, 1)
fitting_mesh.compute_vertex_normals()
fitting_mesh.paint_uniform_color([0.7, 0.7, 0.7])
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([fitting_mesh])

TriangleMesh with 22314 points and 40000 triangles.
22314


In [102]:
deformed_meshes = []
all_meshes = o3d.geometry.TriangleMesh()
for i in range(5,10):
    mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\output/deformed_{i:03}.obj')
    mesh.compute_vertex_normals()
    #print(mesh)
    deformed_meshes.append(mesh)
    all_meshes += mesh
o3d.visualization.draw_geometries([all_meshes])
print(all_meshes, all_meshes.has_vertex_normals())
reference_pcd = o3d.geometry.PointCloud()
reference_pcd.points = o3d.utility.Vector3dVector(all_meshes.vertices)
reference_pcd.normals = o3d.utility.Vector3dVector(all_meshes.vertex_normals)
print(reference_pcd, reference_pcd.has_normals())

TriangleMesh with 98617 points and 196912 triangles. True
PointCloud with 98617 points. True


In [123]:
GoF = 5
key = 0
fitting_meshes = []
for i in range(0,GoF):
    if i==key:
        continue
    decimated_mesh_i = o3d.geometry.TriangleMesh.simplify_quadric_decimation(deformed_meshes[i], 10000, boundary_weight= 8000)
    print(decimated_mesh_i)
    fitting_mesh_i = subdivide_surface_fitting(decimated_mesh_i, deformed_meshes[key], 1)
    print(fitting_mesh_i)
    fitting_mesh_i.compute_vertex_normals()
    fitting_meshes.append(fitting_mesh_i)
fitting_meshes.append(deformed_meshes[key])

TriangleMesh with 5035 points and 9999 triangles.
TriangleMesh with 20072 points and 39996 triangles.
TriangleMesh with 20072 points and 39996 triangles.
TriangleMesh with 5038 points and 10000 triangles.
TriangleMesh with 20080 points and 40000 triangles.
TriangleMesh with 20080 points and 40000 triangles.
TriangleMesh with 5042 points and 9999 triangles.
TriangleMesh with 20088 points and 39996 triangles.
TriangleMesh with 20088 points and 39996 triangles.
TriangleMesh with 5041 points and 10000 triangles.
TriangleMesh with 20087 points and 40000 triangles.
TriangleMesh with 20087 points and 40000 triangles.


In [132]:
deformed_meshes = []
all_meshes = o3d.geometry.TriangleMesh()
for i in range(0,GoF):
    all_meshes += fitting_meshes[i]
#o3d.visualization.draw_geometries([all_meshes])
print(all_meshes, all_meshes.has_vertex_normals())
reference_pcd = o3d.geometry.PointCloud()
reference_pcd.points = o3d.utility.Vector3dVector(all_meshes.vertices)
reference_pcd.normals = o3d.utility.Vector3dVector(all_meshes.vertex_normals)
print(reference_pcd, reference_pcd.has_normals())
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    pre_reference_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(reference_pcd, depth=9, linear_fit=True)
print(mesh)
pre_reference_mesh.compute_vertex_normals()

pre_reference_mesh.paint_uniform_color([0.7, 0.7, 0.7])
o3d.visualization.draw_geometries([pre_reference_mesh])

TriangleMesh with 100051 points and 199372 triangles. True
PointCloud with 100051 points. True
run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 100051 / 18528
[Open3D DEBUG] #   Got kernel density: 0.0120001 (s), 433.355 (MB) / 433.355 (MB) / 603 (MB)
[Open3D DEBUG] #     Got normal field: 0.236 (s), 435.086 (MB) / 435.086 (MB) / 603 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 7.589807e-06 / 7.593678e-01
[Open3D DEBUG] #       Finalized tree: 0.151 (s), 464.398 (MB) / 464.398 (MB) / 603 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.0870001 (s), 447.199 (MB) / 464.398 (MB) / 603 (MB)
[Open3D DEBUG] #Set point constraints: 0.027 (s), 445.844 (MB) / 464.398 (MB) / 603 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 839231 / 955032 / 4089
[Open3D DEBUG] Memory Usage: 445.844 MB
[Open3D DEBUG] # Linear system solved: 0.344 (s), 471.207 (MB) / 471.207 (MB) / 603 (MB)
[Open3D DEBUG] Got average: 0.00300002 (s), 450.488 (MB) / 471.207 (MB) / 603 (MB

In [133]:
reference_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(pre_reference_mesh, 40000, boundary_weight= 8000)
print(reference_mesh)
reference_mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([reference_mesh])

TriangleMesh with 20008 points and 40000 triangles.
